In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM

In [6]:
X_train = joblib.load('../../preprocessed_data/CICIOT/X_train.joblib')
X_test = joblib.load('../../preprocessed_data/CICIOT/X_test.joblib')
Y_train = joblib.load('../../preprocessed_data/CICIOT/Y_train.joblib')
Y_test = joblib.load('../../preprocessed_data/CICIOT/Y_test.joblib')

### CNN training

In [7]:
CNN_model = Sequential([
    Input(shape=(X_train.shape[1], 1)),
    Conv1D(32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2), # keep the maximum each 2 values (divide by 2 the number of values)
    Dropout(0.3), # disable 30% of neurons => reduce overfitting
    Flatten(), # transform the structure from 3D to 2D
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid') # binary output (0=normal or 1=attack)
])

CNN_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

CNN_model.summary()

history = CNN_model.fit(X_train, Y_train, epochs=10, batch_size=64, validation_data=(X_test, Y_test))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 30, 32)            128       
                                                                 
 max_pooling1d (MaxPooling1  (None, 15, 32)            0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 15, 32)            0         
                                                                 
 flatten (Flatten)           (None, 480)               0         
                                                                 
 dense (Dense)               (None, 64)                30784     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                        

KeyboardInterrupt: 

### LSTM training

In [ ]:
LSTM_model = Sequential([
    Input(shape=(X_train.shape[1], 1)),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

LSTM_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

LSTM_model.summary()


history = LSTM_model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_data=(X_test, Y_test))


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 21121 (82.50 KB)
Trainable params: 21121 (82.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
11530/11

### AE-LSTM

In [ ]:
input_dim = X_train.shape[1]  # n_features

# encoder
inputs = Input(shape=(input_dim, 1))
encoded = LSTM(64, return_sequences=False)(inputs)
encoded = Dense(32, activation='relu')(encoded)

# classifier
x = Dropout(0.3)(encoded)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

AE_LSTM_model = Model(inputs, output)

AE_LSTM_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
AE_LSTM_model.summary()


history = AE_LSTM_model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_data=(X_test, Y_test)
)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 33, 1)]           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                16896     
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_5 (Dense)             (None, 64)                2112      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                 65    

Let's save our models

In [ ]:
CNN_model.save("../models/cnn_ciciot_model.keras")

In [ ]:
LSTM_model.save("../models/lstm_ciciot_model.keras")

In [ ]:
AE_LSTM_model.save("../models/ae-lstm_ciciot_model.keras")

In [ ]:
import pickle

In [ ]:
file_name_cnn = '../models/cnn_CICIOT2023.pkl' 
joblib.dump(CNN_model, file_name_cnn) 

file_name_lstm = '../models/lstm_CICIOT2023.pkl' 
joblib.dump(LSTM_model, file_name_lstm) 

file_name_ae_lstm = '../models/ae-lstm_CICIOT2023.pkl' 
joblib.dump(AE_LSTM_model, file_name_ae_lstm)

['../models/ae-lstm_CICIDS2017.pkl']

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred_prob = CNN_model.predict(X_test)


y_pred = (y_pred_prob > 0.5).astype("int32")

# (precision, recall, F1-score)
print(classification_report(Y_test, y_pred, labels=[0,1], target_names=["Normal", "Attacks"]))

# confusion Matrix
cm = confusion_matrix(Y_test, y_pred, labels=[0,1],)
print("Confusion matrix :\n", cm)

# displaying the attack number
tn, fp, fn, tp = cm.ravel()
print(f"Attacks detected (True Positive) : {tp}")
print(f"Normal traffic detected (True Negative) : {tn}")
print(f"Attacks missed (False Negative) : {fn}")
print(f"False alarms (False Positive) : {fp}")


9883/9883 [==============================] - 5s 540us/step
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00    155152
     Attacks       1.00      1.00      1.00    161074

    accuracy                           1.00    316226
   macro avg       1.00      1.00      1.00    316226
weighted avg       1.00      1.00      1.00    316226

Confusion matrix :
 [[155152      0]
 [     1 161073]]
Attacks detected (True Positive) : 161073
Normal traffic detected (True Negative) : 155152
Attacks missed (False Negative) : 1
False alarms (False Positive) : 0
